In [2]:
import pandas as pd
import numpy as np  # 确保已导入numpy
import os

CONFIG = {
    "selection_result_path": r'./short_term_selection_optimized.csv',
    "raw_data_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_result_path": r'./backtest_stable.csv',
    "backtest_summary_path": r'./backtest_stable_summary.txt',
    "log_path": r'./backtest_stable_log.txt',
    "trade_rule": {
        "buy_delay": 1,    # T+1买入
        "sell_delay": 5,   # T+5卖出
        "min_valid_days": 6
    }
}

# --------------------------
# 工具函数（核心修复：替换round为np.round）
# --------------------------
def init_environment():
    with open(CONFIG["log_path"], 'w', encoding='utf-8') as f:
        f.write(f"【稳定版回测启动】{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_msg("✅ 回测环境初始化完成")

def log_msg(msg):
    timestamp = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{timestamp}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def calc_group_stats(df):
    """修复：用np.round兼容float类型，解决round报错"""
    # 1. 处理竞价得分分组
    if 'auction_score' in df.columns:
        try:
            df['auction_group'] = pd.qcut(
                df['auction_score'], 
                q=3, 
                labels=['低竞价得分', '中竞价得分', '高竞价得分'],
                duplicates='drop'
            )
        except:
            df['auction_group'] = pd.cut(
                df['auction_score'],
                bins=[-0.1, 33, 66, 100.1],
                labels=['低竞价得分', '中竞价得分', '高竞价得分']
            )
    else:
        df['auction_group'] = '无数据'
    
    # 2. 处理量价得分分组
    if 'price_volume_score' in df.columns:
        try:
            df['pv_group'] = pd.qcut(
                df['price_volume_score'],
                q=3,
                labels=['低量价得分', '中量价得分', '高量价得分'],
                duplicates='drop'
            )
        except:
            df['pv_group'] = pd.cut(
                df['price_volume_score'],
                bins=[-0.1, 33, 66, 100.1],
                labels=['低量价得分', '中量价得分', '高量价得分']
            )
    else:
        df['pv_group'] = '无数据'
    
    # 3. 分组统计（核心修复：用np.round替代round）
    group_stats = []
    # 竞价得分分组统计
    auction_stats = df.groupby('auction_group', observed=True).agg({
        # 替换x.mean().round(2) → np.round(x.mean(), 2)
        'return_rate': [
            'count', 
            lambda x: np.round(x.mean(), 2),  # 修复：用np.round兼容float
            lambda x: np.round(x.median(), 2),  # 修复
            lambda x: np.round((x>0).mean()*100, 2)  # 修复
        ],
        'stock_code': lambda x: x.nunique()
    })
    auction_stats.columns = ['交易数', '平均收益(%)', '中位数收益(%)', '正收益比例(%)', '股票数']
    for group in auction_stats.index:
        group_stats.append({
            '分组类型': '竞价得分',
            '分组': group,
            '交易数': auction_stats.loc[group, '交易数'],
            '平均收益(%)': auction_stats.loc[group, '平均收益(%)'],
            '中位数收益(%)': auction_stats.loc[group, '中位数收益(%)'],
            '正收益比例(%)': auction_stats.loc[group, '正收益比例(%)']
        })
    
    # 量价得分分组统计（同逻辑修复）
    pv_stats = df.groupby('pv_group', observed=True).agg({
        'return_rate': [
            'count', 
            lambda x: np.round(x.mean(), 2),  # 修复
            lambda x: np.round(x.median(), 2),  # 修复
            lambda x: np.round((x>0).mean()*100, 2)  # 修复
        ],
        'stock_code': lambda x: x.nunique()
    })
    pv_stats.columns = ['交易数', '平均收益(%)', '中位数收益(%)', '正收益比例(%)', '股票数']
    for group in pv_stats.index:
        group_stats.append({
            '分组类型': '量价得分',
            '分组': group,
            '交易数': pv_stats.loc[group, '交易数'],
            '平均收益(%)': pv_stats.loc[group, '平均收益(%)'],
            '中位数收益(%)': pv_stats.loc[group, '中位数收益(%)'],
            '正收益比例(%)': pv_stats.loc[group, '正收益比例(%)']
        })
    
    return pd.DataFrame(group_stats), df

def save_summary(summary_dict, group_stats_df):
    """保存汇总结果"""
    group_text = "\n4. 分组统计\n"
    group_text += "="*60 + "\n"
    group_text += group_stats_df.to_string(index=False, na_rep='-') + "\n"
    
    # 汇总内容中的数值也用np.round统一精度
    summary_content = f"""
【稳定版回测结果】
==========================
回测规则：T日选股 → T+{CONFIG['trade_rule']['buy_delay']}买入 → T+{CONFIG['trade_rule']['sell_delay']}卖出
==========================
1. 基础统计
   - 选股总记录：{summary_dict['total_selection']} 条
   - 有效交易：{summary_dict['valid_trade']} 条
   - 无效交易：{summary_dict['invalid_trade']} 条
   - 有效率：{np.round(summary_dict['valid_rate'], 2)}%

2. 收益统计
   - 平均收益率：{np.round(summary_dict['avg_return'], 2)}%
   - 中位数收益率：{np.round(summary_dict['median_return'], 2)}%
   - 正收益比例：{np.round(summary_dict['positive_ratio'], 2)}%（{summary_dict['positive_count']}/{summary_dict['valid_trade']}）
   - 最大收益：{np.round(summary_dict['max_return'], 2)}%
   - 最小收益：{np.round(summary_dict['min_return'], 2)}%

3. 风险统计
   - 收益标准差：{np.round(summary_dict['std_return'], 2)}%
   - 最大回撤：{np.round(summary_dict['max_drawdown'], 2)}%（简化计算）
{group_text}
=========================="""
    with open(CONFIG["backtest_summary_path"], 'w', encoding='utf-8') as f:
        f.write(summary_content)
    log_msg(f"✅ 回测汇总保存：{CONFIG['backtest_summary_path']}")

# --------------------------
# 主回测逻辑（保持不变）
# --------------------------
def run_backtest():
    try:
        init_environment()
        
        # 1. 加载数据
        log_msg("加载选股结果...")
        selection_df = pd.read_csv(CONFIG["selection_result_path"])
        selection_df['date'] = pd.to_datetime(selection_df['date']).dt.date
        log_msg(f"✅ 选股结果：{len(selection_df)}条记录，{selection_df['stock_code'].nunique()}只股票")
        
        log_msg("加载行情数据...")
        raw_df = pd.read_parquet(CONFIG["raw_data_path"])
        raw_df['date'] = pd.to_datetime(raw_df['date']).dt.date
        raw_df = raw_df[['stock_code', 'date', 'close']].dropna(subset=['close'])
        raw_df = raw_df.sort_values(['stock_code', 'date']).reset_index(drop=True)
        
        # 2. 匹配买卖价格
        log_msg("匹配买卖价格...")
        raw_df['trade_seq'] = raw_df.groupby('stock_code').cumcount()
        selection_df = selection_df.merge(
            raw_df[['stock_code', 'date', 'trade_seq']],
            on=['stock_code', 'date'],
            how='left'
        ).dropna(subset=['trade_seq'])
        selection_df['trade_seq'] = selection_df['trade_seq'].astype(int)
        
        buy_seq = selection_df['trade_seq'] + CONFIG['trade_rule']['buy_delay']
        sell_seq = selection_df['trade_seq'] + CONFIG['trade_rule']['sell_delay']
        
        buy_price = raw_df.set_index(['stock_code', 'trade_seq'])['close'].reindex(
            pd.MultiIndex.from_arrays([selection_df['stock_code'], buy_seq], names=['stock_code', 'trade_seq'])
        ).values
        selection_df['buy_price'] = buy_price
        
        sell_price = raw_df.set_index(['stock_code', 'trade_seq'])['close'].reindex(
            pd.MultiIndex.from_arrays([selection_df['stock_code'], sell_seq], names=['stock_code', 'trade_seq'])
        ).values
        selection_df['sell_price'] = sell_price
        
        # 3. 计算收益
        log_msg("计算收益...")
        selection_df['return_rate'] = (selection_df['sell_price'] - selection_df['buy_price']) / \
                                    selection_df['buy_price'].replace(0, 0.0001) * 100
        valid_mask = selection_df['buy_price'].notna() & selection_df['sell_price'].notna()
        backtest_result = selection_df[valid_mask].copy()
        invalid_count = len(selection_df) - len(backtest_result)
        
        # 4. 分组统计（修复后可正常执行）
        group_stats_df, backtest_result_with_group = calc_group_stats(backtest_result)
        
        # 5. 汇总统计
        if len(backtest_result) > 0:
            summary_dict = {
                "total_selection": len(selection_df),
                "valid_trade": len(backtest_result),
                "invalid_trade": invalid_count,
                "valid_rate": len(backtest_result)/len(selection_df)*100,
                "avg_return": backtest_result['return_rate'].mean(),
                "median_return": backtest_result['return_rate'].median(),
                "positive_count": (backtest_result['return_rate']>0).sum(),
                "positive_ratio": (backtest_result['return_rate']>0).mean()*100,
                "max_return": backtest_result['return_rate'].max(),
                "min_return": backtest_result['return_rate'].min(),
                "std_return": backtest_result['return_rate'].std(),
                "max_drawdown": 0
            }
        else:
            summary_dict = {k: 0 for k in ["total_selection", "valid_trade", "invalid_trade", "valid_rate", "avg_return", "median_return", "positive_count", "positive_ratio", "max_return", "min_return", "std_return", "max_drawdown"]}
        
        # 6. 保存结果
        backtest_result_with_group.to_csv(CONFIG["backtest_result_path"], index=False, encoding='utf-8-sig')
        save_summary(summary_dict, group_stats_df)
        
        # 打印结果
        log_msg(f"\n" + "="*60)
        log_msg(f"✅ 稳定版回测完成！核心结果：")
        log_msg(f"📊 有效交易：{summary_dict['valid_trade']}条 | 正收益比例：{np.round(summary_dict['positive_ratio'], 2)}%")
        log_msg(f"📈 平均收益：{np.round(summary_dict['avg_return'], 2)}% | 中位数收益：{np.round(summary_dict['median_return'], 2)}%")
        log_msg(f"📁 明细：{CONFIG['backtest_result_path']} | 汇总：{CONFIG['backtest_summary_path']}")
        log_msg("="*60)
    
    except Exception as e:
        log_msg(f"❌ 回测失败：{str(e)}")
        raise

if __name__ == "__main__":
    run_backtest()

[2025-10-24 23:41:51] ✅ 回测环境初始化完成
[2025-10-24 23:41:51] 加载选股结果...
[2025-10-24 23:41:51] ✅ 选股结果：630条记录，548只股票
[2025-10-24 23:41:51] 加载行情数据...
[2025-10-24 23:42:11] 匹配买卖价格...
[2025-10-24 23:42:15] 计算收益...
[2025-10-24 23:42:15] ✅ 回测汇总保存：./backtest_stable_summary.txt
[2025-10-24 23:42:15] 
[2025-10-24 23:42:15] ✅ 稳定版回测完成！核心结果：
[2025-10-24 23:42:15] 📊 有效交易：628条 | 正收益比例：50.32%
[2025-10-24 23:42:16] 📈 平均收益：1.41% | 中位数收益：0.1%
[2025-10-24 23:42:16] 📁 明细：./backtest_stable.csv | 汇总：./backtest_stable_summary.txt
[2025-10-24 23:42:16] ============================================================


In [3]:
import pandas as pd
import numpy as np
import os

CONFIG = {
    "selection_result_path": r'./short_term_selection_optimized.csv',
    "raw_data_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_result_path": r'./backtest_stable.csv',
    "backtest_summary_path": r'./backtest_stable_summary.txt',
    "fund_growth_path": r'./fund_growth.csv',  # 新增：资金增长率结果保存路径
    "log_path": r'./backtest_stable_log.txt',
    "trade_rule": {
        "buy_delay": 1,    # T+1买入
        "sell_delay": 5,   # T+5卖出
        "min_valid_days": 6
    },
    "initial_fund": 100000  # 初始资金（可自定义）
}

# --------------------------
# 工具函数（新增资金增长计算相关）
# --------------------------
def init_environment():
    with open(CONFIG["log_path"], 'w', encoding='utf-8') as f:
        f.write(f"【稳定版回测启动】{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_msg("✅ 回测环境初始化完成")

def log_msg(msg):
    timestamp = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{timestamp}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def calc_group_stats(df):
    """修复：用np.round兼容float类型，解决round报错"""
    # 1. 处理竞价得分分组
    if 'auction_score' in df.columns:
        try:
            df['auction_group'] = pd.qcut(
                df['auction_score'], 
                q=3, 
                labels=['低竞价得分', '中竞价得分', '高竞价得分'],
                duplicates='drop'
            )
        except:
            df['auction_group'] = pd.cut(
                df['auction_score'],
                bins=[-0.1, 33, 66, 100.1],
                labels=['低竞价得分', '中竞价得分', '高竞价得分']
            )
    else:
        df['auction_group'] = '无数据'
    
    # 2. 处理量价得分分组
    if 'price_volume_score' in df.columns:
        try:
            df['pv_group'] = pd.qcut(
                df['price_volume_score'],
                q=3,
                labels=['低量价得分', '中量价得分', '高量价得分'],
                duplicates='drop'
            )
        except:
            df['pv_group'] = pd.cut(
                df['price_volume_score'],
                bins=[-0.1, 33, 66, 100.1],
                labels=['低量价得分', '中量价得分', '高量价得分']
            )
    else:
        df['pv_group'] = '无数据'
    
    # 3. 分组统计（核心修复：用np.round替代round）
    group_stats = []
    # 竞价得分分组统计
    auction_stats = df.groupby('auction_group', observed=True).agg({
        'return_rate': [
            'count', 
            lambda x: np.round(x.mean(), 2),
            lambda x: np.round(x.median(), 2),
            lambda x: np.round((x>0).mean()*100, 2)
        ],
        'stock_code': lambda x: x.nunique()
    })
    auction_stats.columns = ['交易数', '平均收益(%)', '中位数收益(%)', '正收益比例(%)', '股票数']
    for group in auction_stats.index:
        group_stats.append({
            '分组类型': '竞价得分',
            '分组': group,
            '交易数': auction_stats.loc[group, '交易数'],
            '平均收益(%)': auction_stats.loc[group, '平均收益(%)'],
            '中位数收益(%)': auction_stats.loc[group, '中位数收益(%)'],
            '正收益比例(%)': auction_stats.loc[group, '正收益比例(%)']
        })
    
    # 量价得分分组统计
    pv_stats = df.groupby('pv_group', observed=True).agg({
        'return_rate': [
            'count', 
            lambda x: np.round(x.mean(), 2),
            lambda x: np.round(x.median(), 2),
            lambda x: np.round((x>0).mean()*100, 2)
        ],
        'stock_code': lambda x: x.nunique()
    })
    pv_stats.columns = ['交易数', '平均收益(%)', '中位数收益(%)', '正收益比例(%)', '股票数']
    for group in pv_stats.index:
        group_stats.append({
            '分组类型': '量价得分',
            '分组': group,
            '交易数': pv_stats.loc[group, '交易数'],
            '平均收益(%)': pv_stats.loc[group, '平均收益(%)'],
            '中位数收益(%)': pv_stats.loc[group, '中位数收益(%)'],
            '正收益比例(%)': pv_stats.loc[group, '正收益比例(%)']
        })
    
    return pd.DataFrame(group_stats), df

# --------------------------
# 新增：按日统计平均收益 + 计算资金增长率
# --------------------------
def calc_daily_return_and_fund_growth(backtest_result):
    """
    1. 按卖出日统计当日所有交易的平均收益
    2. 按公式：当日资金增长率 = 1 + 0.5 * 当日平均收益（%转小数）
    3. 连乘计算累计资金
    """
    log_msg("开始计算每日平均收益和资金增长...")
    
    # 1. 按卖出日分组，计算每日平均收益
    # 先转换卖出日格式（确保是date类型）
    backtest_result['sell_date'] = pd.to_datetime(backtest_result['sell_date']).dt.date
    # 按卖出日统计
    daily_return = backtest_result.groupby('sell_date').agg({
        'return_rate': ['mean', 'count'],  # 当日平均收益、当日交易数
        'return_rate': lambda x: np.round(x.mean(), 4)  # 保留4位小数的平均收益
    }).reset_index()
    daily_return.columns = ['sell_date', 'daily_avg_return']  # 重命名列
    # 过滤当日交易数<2的异常数据（避免单条交易影响）
    daily_return = daily_return[backtest_result.groupby('sell_date')['return_rate'].count().values >= 2].reset_index(drop=True)
    
    # 2. 计算每日资金增长率（公式：1 + 0.5 * 当日平均收益/100）
    daily_return['daily_growth_rate'] = 1 + 0.5 * (daily_return['daily_avg_return'] / 100)
    # 按日期排序（确保连乘顺序正确）
    daily_return = daily_return.sort_values('sell_date').reset_index(drop=True)
    
    # 3. 连乘计算累计资金
    # 初始资金
    daily_return['cumulative_fund'] = CONFIG["initial_fund"]
    # 从第2行开始连乘（第1行累计资金=初始资金*当日增长率）
    for i in range(len(daily_return)):
        if i == 0:
            daily_return.loc[i, 'cumulative_fund'] = CONFIG["initial_fund"] * daily_return.loc[i, 'daily_growth_rate']
        else:
            daily_return.loc[i, 'cumulative_fund'] = daily_return.loc[i-1, 'cumulative_fund'] * daily_return.loc[i, 'daily_growth_rate']
    
    # 格式化数值（保留2位小数）
    daily_return['daily_avg_return'] = np.round(daily_return['daily_avg_return'], 2)
    daily_return['daily_growth_rate'] = np.round(daily_return['daily_growth_rate'], 4)
    daily_return['cumulative_fund'] = np.round(daily_return['cumulative_fund'], 2)
    
    log_msg(f"✅ 每日收益和资金增长计算完成：共{len(daily_return)}个有效交易日")
    return daily_return

# --------------------------
# 修改：保存汇总结果（新增资金增长部分）
# --------------------------
def save_summary(summary_dict, group_stats_df, daily_return):
    """保存汇总结果（新增资金增长统计）"""
    # 资金增长核心指标
    total_trading_days = len(daily_return)
    final_fund = daily_return['cumulative_fund'].iloc[-1] if total_trading_days > 0 else CONFIG["initial_fund"]
    total_return_rate = (final_fund - CONFIG["initial_fund"]) / CONFIG["initial_fund"] * 100
    avg_daily_growth = (final_fund / CONFIG["initial_fund"]) ** (1 / total_trading_days) - 1 if total_trading_days > 0 else 0
    
    # 分组统计文本
    group_text = "\n4. 分组统计\n"
    group_text += "="*60 + "\n"
    group_text += group_stats_df.to_string(index=False, na_rep='-') + "\n"
    
    # 资金增长统计文本
    fund_text = "\n5. 资金增长统计\n"
    fund_text += "="*60 + "\n"
    fund_text += f"初始资金：{CONFIG['initial_fund']:.2f}元\n"
    fund_text += f"最终资金：{final_fund:.2f}元\n"
    fund_text += f"累计收益率：{np.round(total_return_rate, 2)}%\n"
    fund_text += f"有效交易天数：{total_trading_days}天\n"
    fund_text += f"日均资金增长率：{np.round(avg_daily_growth*100, 4)}%\n"
    fund_text += f"最高累计资金：{daily_return['cumulative_fund'].max():.2f}元\n"
    fund_text += f"最低累计资金：{daily_return['cumulative_fund'].min():.2f}元\n"
    
    # 汇总内容
    summary_content = f"""
【稳定版回测结果】
==========================
回测规则：T日选股 → T+{CONFIG['trade_rule']['buy_delay']}买入 → T+{CONFIG['trade_rule']['sell_delay']}卖出
==========================
1. 基础统计
   - 选股总记录：{summary_dict['total_selection']} 条
   - 有效交易：{summary_dict['valid_trade']} 条
   - 无效交易：{summary_dict['invalid_trade']} 条
   - 有效率：{np.round(summary_dict['valid_rate'], 2)}%

2. 收益统计
   - 平均收益率：{np.round(summary_dict['avg_return'], 2)}%
   - 中位数收益率：{np.round(summary_dict['median_return'], 2)}%
   - 正收益比例：{np.round(summary_dict['positive_ratio'], 2)}%（{summary_dict['positive_count']}/{summary_dict['valid_trade']}）
   - 最大收益：{np.round(summary_dict['max_return'], 2)}%
   - 最小收益：{np.round(summary_dict['min_return'], 2)}%

3. 风险统计
   - 收益标准差：{np.round(summary_dict['std_return'], 2)}%
   - 最大回撤：{np.round(summary_dict['max_drawdown'], 2)}%（简化计算）
{group_text}{fund_text}
=========================="""
    with open(CONFIG["backtest_summary_path"], 'w', encoding='utf-8') as f:
        f.write(summary_content)
    # 保存每日资金增长明细
    daily_return.to_csv(CONFIG["fund_growth_path"], index=False, encoding='utf-8-sig')
    log_msg(f"✅ 回测汇总保存：{CONFIG['backtest_summary_path']}")
    log_msg(f"✅ 每日资金增长明细保存：{CONFIG['fund_growth_path']}")

# --------------------------
# 主回测逻辑（新增卖出日计算+资金增长调用）
# --------------------------
def run_backtest():
    try:
        init_environment()
        
        # 1. 加载数据
        log_msg("加载选股结果...")
        selection_df = pd.read_csv(CONFIG["selection_result_path"])
        selection_df['date'] = pd.to_datetime(selection_df['date']).dt.date  # T日（选股日）
        log_msg(f"✅ 选股结果：{len(selection_df)}条记录，{selection_df['stock_code'].nunique()}只股票")
        
        log_msg("加载行情数据...")
        raw_df = pd.read_parquet(CONFIG["raw_data_path"])
        raw_df['date'] = pd.to_datetime(raw_df['date']).dt.date
        raw_df = raw_df[['stock_code', 'date', 'close']].dropna(subset=['close'])
        raw_df = raw_df.sort_values(['stock_code', 'date']).reset_index(drop=True)
        # 新增：为行情数据添加交易序列（用于计算卖出日）
        raw_df['trade_seq'] = raw_df.groupby('stock_code').cumcount()
        # 构建序列→日期映射（用于根据sell_seq获取卖出日）
        seq_date_map = raw_df.set_index(['stock_code', 'trade_seq'])['date'].to_dict()
        
        # 2. 匹配买卖价格+计算卖出日
        log_msg("匹配买卖价格和卖出日...")
        # 合并选股日的交易序列
        selection_df = selection_df.merge(
            raw_df[['stock_code', 'date', 'trade_seq']],
            on=['stock_code', 'date'],
            how='left'
        ).dropna(subset=['trade_seq'])
        selection_df['trade_seq'] = selection_df['trade_seq'].astype(int)
        
        # 计算买卖序列和价格
        buy_seq = selection_df['trade_seq'] + CONFIG['trade_rule']['buy_delay']
        sell_seq = selection_df['trade_seq'] + CONFIG['trade_rule']['sell_delay']
        
        # 匹配买入价、卖出价
        price_map = raw_df.set_index(['stock_code', 'trade_seq'])['close'].to_dict()
        selection_df['buy_price'] = [price_map.get((code, seq), np.nan) for code, seq in zip(selection_df['stock_code'], buy_seq)]
        selection_df['sell_price'] = [price_map.get((code, seq), np.nan) for code, seq in zip(selection_df['stock_code'], sell_seq)]
        
        # 新增：匹配卖出日（用于按日统计收益）
        selection_df['sell_date'] = [seq_date_map.get((code, seq), np.nan) for code, seq in zip(selection_df['stock_code'], sell_seq)]
        
        # 3. 计算收益
        log_msg("计算收益...")
        selection_df['return_rate'] = (selection_df['sell_price'] - selection_df['buy_price']) / \
                                    selection_df['buy_price'].replace(0, 0.0001) * 100
        valid_mask = selection_df['buy_price'].notna() & selection_df['sell_price'].notna() & selection_df['sell_date'].notna()
        backtest_result = selection_df[valid_mask].copy()
        invalid_count = len(selection_df) - len(backtest_result)
        
        # 4. 分组统计
        group_stats_df, backtest_result_with_group = calc_group_stats(backtest_result)
        
        # 5. 新增：计算每日收益和资金增长
        daily_return = calc_daily_return_and_fund_growth(backtest_result)
        
        # 6. 汇总统计
        if len(backtest_result) > 0:
            # 计算最大回撤（简化：基于累计资金的回撤）
            if len(daily_return) > 0:
                cumulative_fund = daily_return['cumulative_fund'].values
                drawdown = [(cumulative_fund[i] - cumulative_fund[:i+1].max()) / cumulative_fund[:i+1].max() for i in range(len(cumulative_fund))]
                max_drawdown = min(drawdown) * 100 if drawdown else 0
            else:
                max_drawdown = 0
            
            summary_dict = {
                "total_selection": len(selection_df),
                "valid_trade": len(backtest_result),
                "invalid_trade": invalid_count,
                "valid_rate": len(backtest_result)/len(selection_df)*100,
                "avg_return": backtest_result['return_rate'].mean(),
                "median_return": backtest_result['return_rate'].median(),
                "positive_count": (backtest_result['return_rate']>0).sum(),
                "positive_ratio": (backtest_result['return_rate']>0).mean()*100,
                "max_return": backtest_result['return_rate'].max(),
                "min_return": backtest_result['return_rate'].min(),
                "std_return": backtest_result['return_rate'].std(),
                "max_drawdown": max_drawdown  # 用累计资金计算的最大回撤
            }
        else:
            summary_dict = {k: 0 for k in ["total_selection", "valid_trade", "invalid_trade", "valid_rate", "avg_return", "median_return", "positive_count", "positive_ratio", "max_return", "min_return", "std_return", "max_drawdown"]}
        
        # 7. 保存结果
        backtest_result_with_group.to_csv(CONFIG["backtest_result_path"], index=False, encoding='utf-8-sig')
        save_summary(summary_dict, group_stats_df, daily_return)
        
        # 打印核心结果
        log_msg(f"\n" + "="*60)
        log_msg(f"✅ 稳定版回测完成！核心结果：")
        log_msg(f"📊 有效交易：{summary_dict['valid_trade']}条 | 正收益比例：{np.round(summary_dict['positive_ratio'], 2)}%")
        log_msg(f"📈 平均收益：{np.round(summary_dict['avg_return'], 2)}% | 累计收益率：{np.round((daily_return['cumulative_fund'].iloc[-1]/CONFIG['initial_fund']-1)*100, 2)}%")
        log_msg(f"💰 初始资金：{CONFIG['initial_fund']}元 | 最终资金：{daily_return['cumulative_fund'].iloc[-1]:.2f}元")
        log_msg(f"📁 明细：{CONFIG['backtest_result_path']} | 资金增长：{CONFIG['fund_growth_path']}")
        log_msg("="*60)
    
    except Exception as e:
        log_msg(f"❌ 回测失败：{str(e)}")
        raise

if __name__ == "__main__":
    run_backtest()

[2025-10-24 23:44:54] ✅ 回测环境初始化完成
[2025-10-24 23:44:54] 加载选股结果...
[2025-10-24 23:44:55] ✅ 选股结果：630条记录，548只股票
[2025-10-24 23:44:55] 加载行情数据...
[2025-10-24 23:45:21] 匹配买卖价格和卖出日...
[2025-10-24 23:45:28] 计算收益...
[2025-10-24 23:45:28] 开始计算每日平均收益和资金增长...


C:\Users\user\AppData\Local\Temp\ipykernel_16112\1126238717.py:153: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '101349.24999999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  daily_return.loc[i, 'cumulative_fund'] = CONFIG["initial_fund"] * daily_return.loc[i, 'daily_growth_rate']


[2025-10-24 23:45:28] ✅ 每日收益和资金增长计算完成：共110个有效交易日
[2025-10-24 23:45:28] ✅ 回测汇总保存：./backtest_stable_summary.txt
[2025-10-24 23:45:28] ✅ 每日资金增长明细保存：./fund_growth.csv
[2025-10-24 23:45:28] 
[2025-10-24 23:45:28] ✅ 稳定版回测完成！核心结果：
[2025-10-24 23:45:28] 📊 有效交易：628条 | 正收益比例：50.32%
[2025-10-24 23:45:28] 📈 平均收益：1.41% | 累计收益率：48.92%
[2025-10-24 23:45:28] 💰 初始资金：100000元 | 最终资金：148916.08元
[2025-10-24 23:45:28] 📁 明细：./backtest_stable.csv | 资金增长：./fund_growth.csv
[2025-10-24 23:45:28] ============================================================
